# Generator danych - Sieć Warsztatów Samochodowych i Sklepów z Akcesoriami

**Scenariusz biznesowy:** Sieć 100 warsztatów samochodowych i sklepów z akcesoriami w całej Polsce.  
**Okres:** 2020-01 do 2024-12 (5 lat)  
**Skala:** ~10-50 GB (konfigurowalna przez SCALE_FACTOR)  

## Tabele:
### Wymiarowe
1. `dim_locations` - lokalizacje (100)
2. `dim_employees` - pracownicy (~2000)
3. `dim_customers` - klienci (~500K)
4. `dim_vehicles` - pojazdy (~600K)
5. `dim_products` - produkty/części (~15K)
6. `dim_services` - katalog usług (~200)
7. `dim_suppliers` - dostawcy (~300)

### Faktowe
8. `fact_work_orders` - zlecenia warsztatowe (~5M)
9. `fact_work_order_items` - pozycje zleceń (~15M)
10. `fact_sales_transactions` - transakcje sklepowe (~30M)
11. `fact_sales_items` - pozycje sprzedaży (~90M)
12. `fact_invoices` - faktury (~35M)
13. `fact_payments` - płatności (~35M)
14. `fact_inventory_movements` - ruchy magazynowe (~50M)

### Wspierające
15. `fact_appointments` - rezerwacje (~5M)
16. `fact_purchase_orders` - zamówienia do dostawców (~500K)
17. `fact_purchase_order_items` - pozycje zamówień (~2M)
18. `fact_customer_feedback` - opinie (~2M)
19. `fact_loyalty_program` - program lojalnościowy (~500K)
20. `fact_employee_schedules` - grafiki pracy (~3M)

In [ ]:
# Instalacja zależności
# !pip install pandas pyarrow faker tqdm

In [ ]:
import os\nimport numpy as np\nimport pandas as pd\nimport pyarrow as pa\nimport pyarrow.parquet as pq\nfrom datetime import datetime, timedelta, date\nfrom faker import Faker\nfrom tqdm import tqdm\nimport random\nimport uuid\nimport gc\nimport json\n\nfake = Faker('pl_PL')\nFaker.seed(42)\nnp.random.seed(42)\nrandom.seed(42)\n\nprint('Biblioteki załadowane OK')

In [ ]:
# ============================================================\n# KONFIGURACJA\n# ============================================================\n\n# SCALE_FACTOR: 1.0 = pełne dane (~30GB), 0.1 = ~3GB, 0.01 = ~300MB do testów\nSCALE_FACTOR = 0.01\n\n# Katalog wyjściowy\nOUTPUT_DIR = './output_data'\n\n# Format: 'parquet' lub 'csv'\nOUTPUT_FORMAT = 'parquet'\n\n# Okres danych\nDATE_START = date(2020, 1, 1)\nDATE_END = date(2024, 12, 31)\n\n# Rozmiar chunka przy zapisie (wiersze)\nCHUNK_SIZE = 200_000\n\n# Liczba lokalizacji\nNUM_LOCATIONS = 100\n\nos.makedirs(OUTPUT_DIR, exist_ok=True)\nprint(f'SCALE_FACTOR = {SCALE_FACTOR}')\nprint(f'OUTPUT_DIR = {OUTPUT_DIR}')\nprint(f'Szacowana wielkość danych: ~{SCALE_FACTOR * 30:.1f} GB')

In [ ]:
# ============================================================\n# HELPERY\n# ============================================================\n\ndef save_table(df, table_name, partition_cols=None):\n    """Zapisuje DataFrame jako parquet lub csv."""\n    table_dir = os.path.join(OUTPUT_DIR, table_name)\n    os.makedirs(table_dir, exist_ok=True)\n    \n    if OUTPUT_FORMAT == 'parquet':\n        if partition_cols:\n            pq.write_to_dataset(\n                pa.Table.from_pandas(df),\n                root_path=table_dir,\n                partition_cols=partition_cols\n            )\n        else:\n            pq.write_table(\n                pa.Table.from_pandas(df),\n                os.path.join(table_dir, f'{table_name}.parquet')\n            )\n    else:\n        df.to_csv(os.path.join(table_dir, f'{table_name}.csv'), index=False)\n    \n    size_mb = df.memory_usage(deep=True).sum() / 1024 / 1024\n    print(f'  ✓ {table_name}: {len(df):,} wierszy, ~{size_mb:.1f} MB w pamięci')\n\n\ndef save_table_chunked(generate_func, table_name, total_rows, partition_cols=None):\n    """Generuje i zapisuje dane w chunkach aby oszczędzić RAM."""\n    table_dir = os.path.join(OUTPUT_DIR, table_name)\n    os.makedirs(table_dir, exist_ok=True)\n    \n    rows_written = 0\n    chunk_num = 0\n    \n    with tqdm(total=total_rows, desc=table_name) as pbar:\n        while rows_written < total_rows:\n            chunk_rows = min(CHUNK_SIZE, total_rows - rows_written)\n            df_chunk = generate_func(chunk_rows, rows_written)\n            \n            if OUTPUT_FORMAT == 'parquet':\n                if partition_cols:\n                    pq.write_to_dataset(\n                        pa.Table.from_pandas(df_chunk),\n                        root_path=table_dir,\n                        partition_cols=partition_cols\n                    )\n                else:\n                    pq.write_table(\n                        pa.Table.from_pandas(df_chunk),\n                        os.path.join(table_dir, f'{table_name}_part{chunk_num:04d}.parquet')\n                    )\n            else:\n                mode = 'w' if chunk_num == 0 else 'a'\n                header = chunk_num == 0\n                df_chunk.to_csv(\n                    os.path.join(table_dir, f'{table_name}.csv'),\n                    index=False, mode=mode, header=header\n                )\n            \n            rows_written += chunk_rows\n            chunk_num += 1\n            pbar.update(chunk_rows)\n            del df_chunk\n            gc.collect()\n    \n    print(f'  ✓ {table_name}: {rows_written:,} wierszy w {chunk_num} chunkach')\n\n\ndef random_dates(start, end, n):\n    """Generuje n losowych dat z zakresu z uwzględnieniem sezonowości."""\n    start_ts = pd.Timestamp(start)\n    end_ts = pd.Timestamp(end)\n    delta = (end_ts - start_ts).days\n    random_days = np.random.randint(0, delta, size=n)\n    dates = start_ts + pd.to_timedelta(random_days, unit='D')\n    return dates\n\n\ndef seasonal_dates(start, end, n):\n    """Generuje daty z sezonowością - więcej w okresach jesień/wiosna."""\n    dates = random_dates(start, end, n)\n    months = dates.month\n    # Wagi sezonowe: więcej w marcu-kwietniu (wymiana opon) i październiku-listopadzie\n    seasonal_weights = {1: 0.7, 2: 0.7, 3: 1.4, 4: 1.4, 5: 1.0, 6: 0.9,\n                        7: 0.8, 8: 0.8, 9: 1.0, 10: 1.4, 11: 1.3, 12: 0.6}\n    weights = np.array([seasonal_weights[m] for m in months])\n    weights = weights / weights.sum()\n    indices = np.random.choice(len(dates), size=n, replace=True, p=weights)\n    return dates[indices]\n\n\ndef generate_uuid_batch(n):\n    """Generuje batch UUID-ów."""\n    return [str(uuid.uuid4()) for _ in range(n)]\n\n\nprint('Helpery załadowane OK')

## 1. Dane referencyjne (słowniki)

In [ ]:
# ============================================================
# SŁOWNIKI DANYCH REFERENCYJNYCH
# ============================================================

WOJEWODZTWA = [
    'dolnośląskie', 'kujawsko-pomorskie', 'lubelskie', 'lubuskie',
    'łódzkie', 'małopolskie', 'mazowieckie', 'opolskie',
    'podkarpackie', 'podlaskie', 'pomorskie', 'śląskie',
    'świętokrzyskie', 'warmińsko-mazurskie', 'wielkopolskie', 'zachodniopomorskie'
]

MIASTA = [
    ('Warszawa', 'mazowieckie', 52.2297, 21.0122),
    ('Kraków', 'małopolskie', 50.0647, 19.9450),
    ('Łódź', 'łódzkie', 51.7592, 19.4560),
    ('Wrocław', 'dolnośląskie', 51.1079, 17.0385),
    ('Poznań', 'wielkopolskie', 52.4064, 16.9252),
    ('Gdańsk', 'pomorskie', 54.3520, 18.6466),
    ('Szczecin', 'zachodniopomorskie', 53.4285, 14.5528),
    ('Bydgoszcz', 'kujawsko-pomorskie', 53.1235, 18.0084),
    ('Lublin', 'lubelskie', 51.2465, 22.5684),
    ('Białystok', 'podlaskie', 53.1325, 23.1688),
    ('Katowice', 'śląskie', 50.2649, 19.0238),
    ('Gdynia', 'pomorskie', 54.5189, 18.5305),
    ('Częstochowa', 'śląskie', 50.8118, 19.1203),
    ('Radom', 'mazowieckie', 51.4027, 21.1471),
    ('Sosnowiec', 'śląskie', 50.2863, 19.1041),
    ('Toruń', 'kujawsko-pomorskie', 53.0138, 18.5984),
    ('Kielce', 'świętokrzyskie', 50.8661, 20.6286),
    ('Rzeszów', 'podkarpackie', 50.0412, 21.9991),
    ('Gliwice', 'śląskie', 50.2945, 18.6714),
    ('Zabrze', 'śląskie', 50.3249, 18.7857),
    ('Olsztyn', 'warmińsko-mazurskie', 53.7784, 20.4801),
    ('Bielsko-Biała', 'śląskie', 49.8224, 19.0586),
    ('Bytom', 'śląskie', 50.3483, 18.9157),
    ('Zielona Góra', 'lubuskie', 51.9356, 15.5062),
    ('Rybnik', 'śląskie', 50.1022, 18.5463),
    ('Ruda Śląska', 'śląskie', 50.2558, 18.8556),
    ('Opole', 'opolskie', 50.6751, 17.9213),
    ('Tychy', 'śląskie', 50.1357, 18.9936),
    ('Gorzów Wielkopolski', 'lubuskie', 52.7325, 15.2369),
    ('Elbląg', 'warmińsko-mazurskie', 54.1522, 19.4088),
    ('Płock', 'mazowieckie', 52.5463, 19.7065),
    ('Dąbrowa Górnicza', 'śląskie', 50.3217, 19.1880),
    ('Wałbrzych', 'dolnośląskie', 50.7714, 16.2843),
    ('Włocławek', 'kujawsko-pomorskie', 52.6483, 19.0677),
    ('Tarnów', 'małopolskie', 50.0121, 20.9858),
    ('Chorzów', 'śląskie', 50.2975, 18.9545),
    ('Koszalin', 'zachodniopomorskie', 54.1943, 16.1715),
    ('Kalisz', 'wielkopolskie', 51.7611, 18.0909),
    ('Legnica', 'dolnośląskie', 51.2070, 16.1619),
    ('Grudziądz', 'kujawsko-pomorskie', 53.4837, 18.7536),
    ('Jaworzno', 'śląskie', 50.2040, 19.2747),
    ('Słupsk', 'pomorskie', 54.4641, 17.0285),
    ('Jastrzębie-Zdrój', 'śląskie', 49.9477, 18.5963),
    ('Nowy Sącz', 'małopolskie', 49.6249, 20.6915),
    ('Jelenia Góra', 'dolnośląskie', 50.9044, 15.7197),
    ('Siedlce', 'mazowieckie', 52.1676, 22.2903),
    ('Mysłowice', 'śląskie', 50.2083, 19.1666),
    ('Konin', 'wielkopolskie', 52.2230, 18.2511),
    ('Piła', 'wielkopolskie', 53.1510, 16.7382),
    ('Piotrków Trybunalski', 'łódzkie', 51.4053, 19.7031),
    ('Inowrocław', 'kujawsko-pomorskie', 52.7936, 18.2614),
    ('Lubin', 'dolnośląskie', 51.4010, 16.2015),
    ('Ostrów Wielkopolski', 'wielkopolskie', 51.6550, 17.8068),
    ('Suwałki', 'podlaskie', 54.1118, 22.9308),
    ('Stargard', 'zachodniopomorskie', 53.3364, 15.0502),
    ('Gniezno', 'wielkopolskie', 52.5348, 17.5827),
    ('Ostrowiec Świętokrzyski', 'świętokrzyskie', 50.9295, 21.3856),
    ('Siemianowice Śląskie', 'śląskie', 50.3264, 19.0296),
    ('Głogów', 'dolnośląskie', 51.6634, 16.0845),
    ('Pabianice', 'łódzkie', 51.6649, 19.3548),
    ('Leszno', 'wielkopolskie', 51.8425, 16.5749),
    ('Żory', 'śląskie', 50.0455, 18.7005),
    ('Pruszków', 'mazowieckie', 52.1707, 20.8120),
    ('Stalowa Wola', 'podkarpackie', 50.5828, 22.0531),
    ('Zamość', 'lubelskie', 50.7230, 23.2519),
    ('Łomża', 'podlaskie', 53.1784, 22.0593),
    ('Mielec', 'podkarpackie', 50.2874, 21.4260),
    ('Tczew', 'pomorskie', 54.0927, 18.7955),
    ('Chełm', 'lubelskie', 51.1431, 23.4716),
    ('Przemyśl', 'podkarpackie', 49.7838, 22.7678),
    ('Starachowice', 'świętokrzyskie', 51.0378, 21.0714),
    ('Wejherowo', 'pomorskie', 54.6059, 18.2354),
    ('Puławy', 'lubelskie', 51.4166, 21.9686),
    ('Skierniewice', 'łódzkie', 51.9542, 20.1576),
    ('Skarżysko-Kamienna', 'świętokrzyskie', 51.1141, 20.8597),
    ('Tarnobrzeg', 'podkarpackie', 50.5731, 21.6792),
    ('Radomsko', 'łódzkie', 51.0671, 19.4462),
    ('Kędzierzyn-Koźle', 'opolskie', 50.3494, 18.2074),
    ('Biała Podlaska', 'lubelskie', 52.0326, 23.1166),
    ('Oświęcim', 'małopolskie', 50.0343, 19.2098),
    ('Sandomierz', 'świętokrzyskie', 50.6827, 21.7489),
    ('Busko-Zdrój', 'świętokrzyskie', 50.4710, 20.7192),
    ('Nowa Sól', 'lubuskie', 51.8063, 15.7146),
    ('Nysa', 'opolskie', 50.4743, 17.3346),
    ('Otwock', 'mazowieckie', 52.1054, 21.2614),
    ('Szczytno', 'warmińsko-mazurskie', 53.5630, 20.9868),
    ('Kutno', 'łódzkie', 52.2318, 19.3569),
    ('Sanok', 'podkarpackie', 49.5566, 22.2059),
    ('Świnoujście', 'zachodniopomorskie', 53.9101, 14.2474),
    ('Świdnica', 'dolnośląskie', 50.8463, 16.4872),
    ('Chojnice', 'pomorskie', 53.6953, 17.5551),
    ('Mińsk Mazowiecki', 'mazowieckie', 52.1790, 21.5617),
    ('Żyrardów', 'mazowieckie', 52.0491, 20.4467),
    ('Wołomin', 'mazowieckie', 52.3461, 21.2405),
    ('Nowy Targ', 'małopolskie', 49.4782, 20.0323),
    ('Giżycko', 'warmińsko-mazurskie', 54.0380, 21.7647),
    ('Brodnica', 'kujawsko-pomorskie', 53.2600, 19.3954),
    ('Bolesławiec', 'dolnośląskie', 51.2622, 15.5694),
    ('Świecie', 'kujawsko-pomorskie', 53.4100, 18.4316),
]

MARKI_SAMOCHODOW = {
    'Toyota': ['Corolla', 'Yaris', 'RAV4', 'Camry', 'C-HR', 'Aygo', 'Hilux', 'Land Cruiser'],
    'Volkswagen': ['Golf', 'Passat', 'Polo', 'Tiguan', 'T-Roc', 'Arteon', 'Touran', 'Caddy'],
    'Skoda': ['Octavia', 'Fabia', 'Superb', 'Kodiaq', 'Karoq', 'Kamiq', 'Scala', 'Citigo'],
    'Ford': ['Focus', 'Fiesta', 'Mondeo', 'Kuga', 'Puma', 'EcoSport', 'Transit', 'Ranger'],
    'Opel': ['Astra', 'Corsa', 'Insignia', 'Mokka', 'Crossland', 'Grandland', 'Combo', 'Vivaro'],
    'BMW': ['Seria 3', 'Seria 5', 'X1', 'X3', 'Seria 1', 'X5', 'Seria 7', 'X6'],
    'Audi': ['A3', 'A4', 'A6', 'Q3', 'Q5', 'A1', 'Q7', 'TT'],
    'Mercedes': ['Klasa A', 'Klasa C', 'Klasa E', 'GLC', 'GLA', 'GLE', 'Klasa S', 'Sprinter'],
    'Renault': ['Clio', 'Megane', 'Captur', 'Kadjar', 'Scenic', 'Kangoo', 'Master', 'Trafic'],
    'Hyundai': ['i30', 'Tucson', 'i20', 'Kona', 'Santa Fe', 'i10', 'ix20', 'Ioniq'],
    'Kia': ['Ceed', 'Sportage', 'Rio', 'Stonic', 'Sorento', 'Picanto', 'XCeed', 'Niro'],
    'Fiat': ['500', 'Tipo', 'Panda', 'Punto', '500X', 'Ducato', 'Doblo', '500L'],
    'Peugeot': ['208', '308', '3008', '2008', '508', '5008', 'Partner', 'Rifter'],
    'Citroen': ['C3', 'C4', 'C5 Aircross', 'Berlingo', 'C3 Aircross', 'C1', 'Jumper', 'Jumpy'],
    'Dacia': ['Duster', 'Sandero', 'Logan', 'Dokker', 'Lodgy', 'Spring'],
    'Nissan': ['Qashqai', 'Juke', 'Micra', 'X-Trail', 'Navara', 'Leaf', 'Note'],
    'Honda': ['Civic', 'CR-V', 'Jazz', 'HR-V', 'Accord', 'e'],
    'Mazda': ['3', '6', 'CX-5', 'CX-3', 'CX-30', 'MX-5', '2'],
    'Volvo': ['XC60', 'XC40', 'V60', 'S60', 'XC90', 'V40', 'S90'],
    'Suzuki': ['Vitara', 'Swift', 'SX4 S-Cross', 'Ignis', 'Jimny', 'Baleno'],
}

# Wagi popularności marek w Polsce (sumują się do ~1)
MARKA_WAGI = {
    'Toyota': 0.12, 'Volkswagen': 0.11, 'Skoda': 0.10, 'Ford': 0.08,
    'Opel': 0.08, 'BMW': 0.05, 'Audi': 0.05, 'Mercedes': 0.04,
    'Renault': 0.06, 'Hyundai': 0.06, 'Kia': 0.06, 'Fiat': 0.04,
    'Peugeot': 0.04, 'Citroen': 0.03, 'Dacia': 0.03, 'Nissan': 0.02,
    'Honda': 0.02, 'Mazda': 0.02, 'Volvo': 0.02, 'Suzuki': 0.02,
}

TYPY_PALIWA = ['benzyna', 'diesel', 'LPG', 'hybryda', 'elektryczny']
PALIWO_WAGI = [0.35, 0.30, 0.15, 0.15, 0.05]

KOLORY = ['biały', 'czarny', 'srebrny', 'szary', 'czerwony', 'niebieski',
           'granatowy', 'zielony', 'brązowy', 'beżowy', 'złoty', 'bordowy']

KATEGORIE_PRODUKTOW = {
    'Oleje i płyny': [
        'Olej silnikowy 5W-30', 'Olej silnikowy 5W-40', 'Olej silnikowy 10W-40',
        'Olej silnikowy 0W-20', 'Płyn hamulcowy DOT4', 'Płyn chłodniczy G12',
        'Płyn do spryskiwaczy letni', 'Płyn do spryskiwaczy zimowy',
        'Olej przekładniowy', 'Płyn do wspomagania', 'Płyn AdBlue 10L',
    ],
    'Filtry': [
        'Filtr oleju', 'Filtr powietrza', 'Filtr kabinowy', 'Filtr paliwa',
        'Filtr kabinowy z węglem aktywnym', 'Filtr DPF', 'Filtr GPF',
    ],
    'Klocki i tarcze hamulcowe': [
        'Klocki hamulcowe przód', 'Klocki hamulcowe tył',
        'Tarcze hamulcowe przód', 'Tarcze hamulcowe tył',
        'Szczęki hamulcowe', 'Bębny hamulcowe',
    ],
    'Opony': [
        'Opona letnia 205/55 R16', 'Opona letnia 195/65 R15',
        'Opona letnia 225/45 R17', 'Opona zimowa 205/55 R16',
        'Opona zimowa 195/65 R15', 'Opona zimowa 225/45 R17',
        'Opona całoroczna 205/55 R16', 'Opona całoroczna 195/65 R15',
    ],
    'Akumulatory': [
        'Akumulator 60Ah', 'Akumulator 70Ah', 'Akumulator 74Ah',
        'Akumulator 80Ah', 'Akumulator 100Ah', 'Akumulator AGM 70Ah',
    ],
    'Oświetlenie': [
        'Żarówka H7', 'Żarówka H4', 'Żarówka H1', 'Żarówka LED H7',
        'Żarówka LED H4', 'Żarówka W5W', 'Żarówka P21W',
        'Żarówka ksenonowa D1S', 'Żarówka ksenonowa D2S',
    ],
    'Wycieraczki': [
        'Wycieraczka przednia lewa', 'Wycieraczka przednia prawa',
        'Wycieraczka tylna', 'Komplet wycieraczek przednich',
    ],
    'Układ zawieszenia': [
        'Amortyzator przedni', 'Amortyzator tylny', 'Sprężyna zawieszenia',
        'Wahacz dolny', 'Łącznik stabilizatora', 'Tuleja wahacza',
        'Końcówka drążka kierowniczego', 'Drążek kierowniczy',
    ],
    'Układ rozrządu': [
        'Pasek rozrządu', 'Zestaw rozrządu z pompą wody',
        'Pasek klinowy wielorowkowy', 'Napinacz paska rozrządu',
        'Łańcuch rozrządu', 'Zestaw łańcucha rozrządu',
    ],
    'Układ wydechowy': [
        'Tłumik końcowy', 'Tłumik środkowy', 'Katalizator',
        'Rura wydechowa', 'Filtr cząstek stałych DPF',
        'Sonda lambda', 'Uszczelka wydechu',
    ],
    'Układ elektryczny': [
        'Alternator', 'Rozrusznik', 'Cewka zapłonowa',
        'Świeca zapłonowa', 'Świeca żarowa', 'Czujnik ABS',
        'Czujnik temperatury', 'Czujnik ciśnienia oleju',
    ],
    'Sprzęgło': [
        'Komplet sprzęgła', 'Tarcza sprzęgła', 'Docisk sprzęgła',
        'Łożysko oporowe sprzęgła', 'Koło dwumasowe',
    ],
    'Chemia samochodowa': [
        'Szampon samochodowy', 'Wosk do lakieru', 'Środek do felg',
        'Odmrażacz do szyb', 'Odświeżacz powietrza', 'Pasta polerska',
        'Środek do czyszczenia tapicerki', 'Silikon do uszczelek',
        'Środek do plastików', 'Preparat antykorozyjny',
    ],
    'Akcesoria': [
        'Dywaniki gumowe komplet', 'Dywaniki welurowe komplet',
        'Pokrowce na fotele', 'Organizer bagażnika', 'Apteczka samochodowa',
        'Trójkąt ostrzegawczy', 'Gaśnica samochodowa', 'Kable rozruchowe',
        'Kompas samochodowy', 'Ładowarka USB do samochodu',
        'Uchwyt na telefon', 'Kamera cofania', 'Czujniki parkowania',
        'Bagażnik dachowy', 'Box dachowy', 'Hak holowniczy',
    ],
    'Narzędzia': [
        'Klucz do kół', 'Podnośnik hydrauliczny', 'Komplet kluczy nasadowych',
        'Klucz dynamometryczny', 'Zestaw naprawczy opon',
    ],
}

KATALOG_USLUG = [
    # (nazwa, kategoria, min_cena_netto, max_cena_netto, czas_min)
    ('Wymiana oleju i filtra', 'Serwis okresowy', 80, 150, 30),
    ('Przegląd okresowy', 'Serwis okresowy', 150, 350, 60),
    ('Wymiana filtra powietrza', 'Serwis okresowy', 30, 60, 15),
    ('Wymiana filtra kabinowego', 'Serwis okresowy', 30, 60, 15),
    ('Wymiana płynu hamulcowego', 'Serwis okresowy', 80, 150, 30),
    ('Wymiana płynu chłodniczego', 'Serwis okresowy', 100, 200, 45),
    ('Wymiana świec zapłonowych', 'Serwis okresowy', 60, 150, 30),
    ('Wymiana świec żarowych', 'Serwis okresowy', 100, 300, 60),
    ('Wymiana klocków hamulcowych przód', 'Hamulce', 100, 200, 45),
    ('Wymiana klocków hamulcowych tył', 'Hamulce', 80, 180, 45),
    ('Wymiana tarcz i klocków przód', 'Hamulce', 200, 400, 60),
    ('Wymiana tarcz i klocków tył', 'Hamulce', 180, 350, 60),
    ('Wymiana szczęk hamulcowych', 'Hamulce', 100, 200, 60),
    ('Wymiana opon (4 szt.)', 'Opony', 80, 160, 45),
    ('Wyważanie kół (4 szt.)', 'Opony', 40, 80, 30),
    ('Przechowywanie opon (sezon)', 'Opony', 60, 120, 15),
    ('Naprawa opony', 'Opony', 20, 50, 20),
    ('Geometria kół', 'Opony', 100, 200, 45),
    ('Wymiana amortyzatorów przód', 'Zawieszenie', 200, 500, 120),
    ('Wymiana amortyzatorów tył', 'Zawieszenie', 150, 400, 90),
    ('Wymiana wahacza', 'Zawieszenie', 150, 350, 90),
    ('Wymiana łącznika stabilizatora', 'Zawieszenie', 50, 120, 30),
    ('Wymiana końcówki drążka', 'Zawieszenie', 80, 180, 45),
    ('Wymiana paska rozrządu', 'Rozrząd', 400, 1200, 240),
    ('Wymiana zestawu rozrządu z pompą', 'Rozrząd', 600, 1800, 300),
    ('Wymiana paska klinowego', 'Rozrząd', 80, 200, 45),
    ('Wymiana sprzęgła', 'Sprzęgło', 500, 1500, 360),
    ('Wymiana koła dwumasowego', 'Sprzęgło', 800, 2500, 420),
    ('Wymiana rozrusznika', 'Układ elektryczny', 200, 500, 90),
    ('Wymiana alternatora', 'Układ elektryczny', 250, 600, 90),
    ('Diagnostyka komputerowa', 'Diagnostyka', 50, 150, 30),
    ('Kasowanie błędów', 'Diagnostyka', 30, 80, 15),
    ('Kontrola klimatyzacji', 'Klimatyzacja', 50, 100, 30),
    ('Serwis klimatyzacji', 'Klimatyzacja', 150, 350, 60),
    ('Ozonowanie wnętrza', 'Klimatyzacja', 50, 100, 30),
    ('Wymiana tłumika', 'Układ wydechowy', 150, 400, 60),
    ('Wymiana katalizatora', 'Układ wydechowy', 500, 2000, 120),
    ('Spawanie wydechu', 'Układ wydechowy', 50, 150, 30),
    ('Wymiana akumulatora', 'Elektryka', 30, 60, 15),
    ('Wymiana żarówki', 'Elektryka', 20, 80, 15),
    ('Lakierowanie elementu', 'Blacharstwo', 300, 1500, 480),
    ('Naprawa blacharsko-lakiernicza', 'Blacharstwo', 500, 5000, 960),
    ('Polerowanie lakieru', 'Blacharstwo', 200, 600, 240),
    ('Usuwanie wgnieceń PDR', 'Blacharstwo', 100, 500, 120),
    ('Przegląd techniczny', 'Przegląd', 99, 99, 30),
    ('Badanie techniczne + spaliny', 'Przegląd', 162, 162, 45),
    ('Mycie silnika', 'Inne', 80, 200, 60),
    ('Zabezpieczenie antykorozyjne podwozia', 'Inne', 200, 600, 120),
]

METODY_PLATNOSCI = ['gotówka', 'karta', 'przelew', 'BLIK', 'leasing', 'raty']
PLATNOSC_WAGI = [0.15, 0.40, 0.20, 0.15, 0.05, 0.05]

STATUSY_ZLECEN = ['nowe', 'w_trakcie', 'oczekuje_na_czesci', 'zakonczone', 'anulowane']
STATUSY_WAGI = [0.02, 0.03, 0.01, 0.92, 0.02]

TYPY_LOKALIZACJI = ['warsztat', 'sklep', 'warsztat_i_sklep']
TYP_LOKALIZACJI_WAGI = [0.30, 0.20, 0.50]

STANOWISKA = {
    'warsztat': ['mechanik', 'mechanik_senior', 'elektryk_samochodowy', 'blacharz', 'lakiernik', 'diagnosta'],
    'sklep': ['sprzedawca', 'sprzedawca_senior', 'kasjer', 'magazynier'],
    'zarzadzanie': ['kierownik_oddzialu', 'zastepca_kierownika', 'ksiegowy'],
}

print(f'Załadowano słowniki:')
print(f'  - {len(MIASTA)} miast')
print(f'  - {len(MARKI_SAMOCHODOW)} marek samochodów')
print(f'  - {sum(len(v) for v in KATEGORIE_PRODUKTOW.values())} produktów w {len(KATEGORIE_PRODUKTOW)} kategoriach')
print(f'  - {len(KATALOG_USLUG)} usług warsztatowych')

## 2. Tabele wymiarowe (dimension tables)

In [ ]:
# ============================================================
# dim_locations - 100 lokalizacji warsztatów/sklepów
# ============================================================
print('Generowanie dim_locations...')

locations = []
for i, (miasto, woj, lat, lon) in enumerate(MIASTA[:NUM_LOCATIONS]):
    typ = np.random.choice(TYPY_LOKALIZACJI, p=TYP_LOKALIZACJI_WAGI)
    otwarcie = fake.date_between(start_date=date(2005, 1, 1), end_date=date(2020, 6, 30))
    locations.append({
        'location_id': i + 1,
        'location_code': f'LOC-{i+1:03d}',
        'nazwa': f'AutoSerwis {miasto}',
        'typ': typ,
        'ulica': fake.street_address(),
        'miasto': miasto,
        'wojewodztwo': woj,
        'kod_pocztowy': fake.postcode(),
        'latitude': lat + np.random.uniform(-0.02, 0.02),
        'longitude': lon + np.random.uniform(-0.02, 0.02),
        'telefon': fake.phone_number(),
        'email': f'serwis.{miasto.lower().replace(" ", "").replace("-", "")}@autoserwis.pl',
        'kierownik_id': None,  # uzupełnimy po generowaniu pracowników
        'liczba_stanowisk': np.random.randint(4, 12) if typ != 'sklep' else 0,
        'powierzchnia_m2': np.random.randint(200, 800),
        'data_otwarcia': otwarcie,
        'czy_aktywna': True if i < 95 else False,  # 5 lokalizacji zamkniętych
    })

df_locations = pd.DataFrame(locations)
save_table(df_locations, 'dim_locations')
df_locations.head()

In [ ]:
# ============================================================
# dim_employees - pracownicy (~20 na lokalizację = ~2000)
# ============================================================
print('Generowanie dim_employees...')

employees = []
emp_id = 1

for _, loc in df_locations.iterrows():
    loc_id = loc['location_id']
    typ = loc['typ']
    
    # Kadra zarządzająca - zawsze
    for stanowisko in STANOWISKA['zarzadzanie']:
        employees.append({
            'employee_id': emp_id,
            'employee_code': f'EMP-{emp_id:05d}',
            'imie': fake.first_name(),
            'nazwisko': fake.last_name(),
            'pesel': fake.pesel(),
            'stanowisko': stanowisko,
            'location_id': loc_id,
            'data_zatrudnienia': fake.date_between(
                start_date=loc['data_otwarcia'],
                end_date=min(loc['data_otwarcia'] + timedelta(days=365), DATE_END)
            ),
            'data_zwolnienia': None,
            'stawka_godzinowa': round(np.random.uniform(45, 80), 2),
            'czy_aktywny': loc['czy_aktywna'],
        })
        emp_id += 1
    
    # Pracownicy warsztatu
    if typ in ('warsztat', 'warsztat_i_sklep'):
        n_mechanikow = np.random.randint(5, 10)
        for _ in range(n_mechanikow):
            stanowisko = random.choice(STANOWISKA['warsztat'])
            employees.append({
                'employee_id': emp_id,
                'employee_code': f'EMP-{emp_id:05d}',
                'imie': fake.first_name_male() if random.random() < 0.9 else fake.first_name_female(),
                'nazwisko': fake.last_name(),
                'pesel': fake.pesel(),
                'stanowisko': stanowisko,
                'location_id': loc_id,
                'data_zatrudnienia': fake.date_between(
                    start_date=loc['data_otwarcia'],
                    end_date=DATE_END
                ),
                'data_zwolnienia': fake.date_between(start_date=date(2022,1,1), end_date=DATE_END) if random.random() < 0.1 else None,
                'stawka_godzinowa': round(np.random.uniform(30, 65), 2),
                'czy_aktywny': random.random() > 0.1,
            })
            emp_id += 1
    
    # Pracownicy sklepu
    if typ in ('sklep', 'warsztat_i_sklep'):
        n_sprzedawcow = np.random.randint(3, 7)
        for _ in range(n_sprzedawcow):
            stanowisko = random.choice(STANOWISKA['sklep'])
            employees.append({
                'employee_id': emp_id,
                'employee_code': f'EMP-{emp_id:05d}',
                'imie': fake.first_name(),
                'nazwisko': fake.last_name(),
                'pesel': fake.pesel(),
                'stanowisko': stanowisko,
                'location_id': loc_id,
                'data_zatrudnienia': fake.date_between(
                    start_date=loc['data_otwarcia'],
                    end_date=DATE_END
                ),
                'data_zwolnienia': fake.date_between(start_date=date(2022,1,1), end_date=DATE_END) if random.random() < 0.15 else None,
                'stawka_godzinowa': round(np.random.uniform(25, 45), 2),
                'czy_aktywny': random.random() > 0.12,
            })
            emp_id += 1

df_employees = pd.DataFrame(employees)
save_table(df_employees, 'dim_employees')

# Lista ID mechaników i sprzedawców do użycia w tabelach faktowych
mechanic_ids = df_employees[df_employees['stanowisko'].isin(STANOWISKA['warsztat'])]['employee_id'].values
seller_ids = df_employees[df_employees['stanowisko'].isin(STANOWISKA['sklep'])]['employee_id'].values

# Mapowanie: location_id -> lista mechaników
loc_mechanics = df_employees[df_employees['stanowisko'].isin(STANOWISKA['warsztat'])].groupby('location_id')['employee_id'].apply(list).to_dict()
loc_sellers = df_employees[df_employees['stanowisko'].isin(STANOWISKA['sklep'])].groupby('location_id')['employee_id'].apply(list).to_dict()

print(f'  Mechanicy: {len(mechanic_ids)}, Sprzedawcy: {len(seller_ids)}')
df_employees.head()

In [ ]:
# ============================================================
# dim_customers - klienci (500K * SCALE_FACTOR)
# ============================================================
NUM_CUSTOMERS = int(500_000 * SCALE_FACTOR)
print(f'Generowanie dim_customers ({NUM_CUSTOMERS:,} klientów)...')

customer_types = np.random.choice(
    ['indywidualny', 'firma'], size=NUM_CUSTOMERS, p=[0.7, 0.3]
)

customers = pd.DataFrame({
    'customer_id': np.arange(1, NUM_CUSTOMERS + 1),
    'customer_code': [f'CUS-{i:07d}' for i in range(1, NUM_CUSTOMERS + 1)],
    'typ_klienta': customer_types,
    'imie': [fake.first_name() if t == 'indywidualny' else '' for t in customer_types],
    'nazwisko': [fake.last_name() if t == 'indywidualny' else '' for t in customer_types],
    'nazwa_firmy': [fake.company() if t == 'firma' else '' for t in customer_types],
    'nip': [fake.company_vat() if t == 'firma' else '' for t in customer_types],
    'email': [fake.email() for _ in range(NUM_CUSTOMERS)],
    'telefon': [fake.phone_number() for _ in range(NUM_CUSTOMERS)],
    'miasto': np.random.choice([m[0] for m in MIASTA], size=NUM_CUSTOMERS),
    'kod_pocztowy': [fake.postcode() for _ in range(NUM_CUSTOMERS)],
    'data_rejestracji': random_dates(DATE_START, DATE_END, NUM_CUSTOMERS),
    'preferowana_lokalizacja_id': np.random.randint(1, NUM_LOCATIONS + 1, size=NUM_CUSTOMERS),
    'zgoda_marketing': np.random.choice([True, False], size=NUM_CUSTOMERS, p=[0.6, 0.4]),
})

df_customers = customers
save_table(df_customers, 'dim_customers')
customer_ids = df_customers['customer_id'].values
print(f'  Klienci indywidualni: {(df_customers["typ_klienta"]=="indywidualny").sum():,}')
print(f'  Klienci firmowi: {(df_customers["typ_klienta"]=="firma").sum():,}')
df_customers.head()

In [ ]:
# ============================================================
# dim_vehicles - pojazdy klientów (600K * SCALE_FACTOR)
# ============================================================
NUM_VEHICLES = int(600_000 * SCALE_FACTOR)
print(f'Generowanie dim_vehicles ({NUM_VEHICLES:,} pojazdów)...')

marki = list(MARKI_SAMOCHODOW.keys())
wagi = [MARKA_WAGI[m] for m in marki]
wagi_norm = np.array(wagi) / sum(wagi)

chosen_marki = np.random.choice(marki, size=NUM_VEHICLES, p=wagi_norm)
chosen_modele = [random.choice(MARKI_SAMOCHODOW[m]) for m in chosen_marki]

vehicles = pd.DataFrame({
    'vehicle_id': np.arange(1, NUM_VEHICLES + 1),
    'customer_id': np.random.choice(customer_ids, size=NUM_VEHICLES),
    'marka': chosen_marki,
    'model': chosen_modele,
    'rocznik': np.random.randint(2005, 2025, size=NUM_VEHICLES),
    'vin': [fake.bothify('???#########??????').upper() for _ in range(NUM_VEHICLES)],
    'nr_rejestracyjny': [fake.license_plate() for _ in range(NUM_VEHICLES)],
    'typ_paliwa': np.random.choice(TYPY_PALIWA, size=NUM_VEHICLES, p=PALIWO_WAGI),
    'pojemnosc_silnika': np.random.choice(
        [1.0, 1.2, 1.4, 1.5, 1.6, 1.8, 2.0, 2.2, 2.5, 3.0],
        size=NUM_VEHICLES,
        p=[0.05, 0.10, 0.15, 0.12, 0.18, 0.12, 0.12, 0.06, 0.05, 0.05]
    ),
    'moc_km': np.random.randint(60, 350, size=NUM_VEHICLES),
    'kolor': np.random.choice(KOLORY, size=NUM_VEHICLES),
    'przebieg_km': np.random.randint(5000, 350000, size=NUM_VEHICLES),
    'data_pierwszej_rejestracji': random_dates(date(2005,1,1), DATE_END, NUM_VEHICLES),
})

df_vehicles = vehicles
save_table(df_vehicles, 'dim_vehicles')
vehicle_ids = df_vehicles['vehicle_id'].values
print(f'  Średnio {NUM_VEHICLES / NUM_CUSTOMERS:.1f} pojazdów na klienta')
df_vehicles.head()

In [ ]:
# ============================================================
# dim_products - produkty/części (~15K z wariantami)
# ============================================================
print('Generowanie dim_products...')

products = []
prod_id = 1

for kategoria, produkty_lista in KATEGORIE_PRODUKTOW.items():
    for nazwa_bazowa in produkty_lista:
        # Każdy produkt ma kilka wariantów (różne marki producenta)
        producenci = random.sample(
            ['Bosch', 'Continental', 'Valeo', 'Hella', 'Mann', 'Mahle', 'NGK',
             'Brembo', 'TRW', 'KYB', 'Monroe', 'Sachs', 'LuK', 'Gates',
             'SKF', 'Dayco', 'Castrol', 'Mobil', 'Shell', 'Total', 'Motul',
             'Liqui Moly', 'K2', 'Meguiars', 'Sonax', 'Goodyear', 'Michelin',
             'Continental', 'Bridgestone', 'Pirelli', 'Varta', 'Exide', 'Banner'],
            k=min(random.randint(2, 6), 32)
        )
        for producent in producenci:
            cena_bazowa = round(np.random.uniform(5, 800), 2)
            # Droższe produkty: opony, akumulatory, sprzęgło
            if 'Opona' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(180, 600), 2)
            elif 'Akumulator' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(250, 800), 2)
            elif 'Komplet sprzęgła' in nazwa_bazowa or 'Koło dwumasowe' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(400, 2000), 2)
            elif 'Amortyzator' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(100, 400), 2)
            elif 'Filtr' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(15, 80), 2)
            elif 'Klocki' in nazwa_bazowa or 'Tarcze' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(60, 300), 2)
            elif 'Olej' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(30, 180), 2)
            elif 'Żarówka' in nazwa_bazowa:
                cena_bazowa = round(np.random.uniform(8, 120), 2)
            
            marza = round(np.random.uniform(1.15, 1.45), 2)
            
            products.append({
                'product_id': prod_id,
                'product_code': f'PRD-{prod_id:06d}',
                'nazwa': f'{nazwa_bazowa} {producent}',
                'kategoria': kategoria,
                'producent': producent,
                'cena_zakupu_netto': cena_bazowa,
                'cena_sprzedazy_netto': round(cena_bazowa * marza, 2),
                'vat_procent': 23,
                'jednostka': 'szt' if 'Olej' not in nazwa_bazowa and 'Płyn' not in nazwa_bazowa else 'L',
                'waga_kg': round(np.random.uniform(0.1, 15), 2),
                'min_stan_magazynowy': np.random.randint(2, 20),
                'czy_aktywny': random.random() > 0.05,
            })
            prod_id += 1

df_products = pd.DataFrame(products)
save_table(df_products, 'dim_products')
product_ids = df_products['product_id'].values
print(f'  Produktów: {len(df_products):,} w {len(KATEGORIE_PRODUKTOW)} kategoriach')
df_products.head()

In [ ]:
# ============================================================
# dim_services - katalog usług warsztatowych
# ============================================================
print('Generowanie dim_services...')

services = []
for i, (nazwa, kategoria, min_c, max_c, czas) in enumerate(KATALOG_USLUG):
    services.append({
        'service_id': i + 1,
        'service_code': f'SRV-{i+1:03d}',
        'nazwa': nazwa,
        'kategoria': kategoria,
        'cena_min_netto': min_c,
        'cena_max_netto': max_c,
        'szacowany_czas_min': czas,
        'czy_aktywna': True,
    })

df_services = pd.DataFrame(services)
save_table(df_services, 'dim_services')
service_ids = df_services['service_id'].values

# ============================================================
# dim_suppliers - dostawcy części
# ============================================================
print('Generowanie dim_suppliers...')

NUM_SUPPLIERS = 300
suppliers = []
for i in range(NUM_SUPPLIERS):
    suppliers.append({
        'supplier_id': i + 1,
        'supplier_code': f'SUP-{i+1:04d}',
        'nazwa': fake.company(),
        'nip': fake.company_vat(),
        'miasto': random.choice([m[0] for m in MIASTA]),
        'adres': fake.street_address(),
        'kod_pocztowy': fake.postcode(),
        'telefon': fake.phone_number(),
        'email': fake.company_email(),
        'osoba_kontaktowa': fake.name(),
        'warunki_platnosci_dni': random.choice([14, 21, 30, 45, 60]),
        'min_wartosc_zamowienia': round(np.random.uniform(200, 2000), 2),
        'czy_aktywny': random.random() > 0.08,
    })

df_suppliers = pd.DataFrame(suppliers)
save_table(df_suppliers, 'dim_suppliers')
supplier_ids = df_suppliers['supplier_id'].values

print(f'\n=== PODSUMOWANIE TABEL WYMIAROWYCH ===')
for name, df in [('dim_locations', df_locations), ('dim_employees', df_employees),
                  ('dim_customers', df_customers), ('dim_vehicles', df_vehicles),
                  ('dim_products', df_products), ('dim_services', df_services),
                  ('dim_suppliers', df_suppliers)]:
    print(f'  {name}: {len(df):,} wierszy')

## 3. Tabele faktowe - zlecenia warsztatowe

In [ ]:
# ============================================================
# fact_work_orders - zlecenia warsztatowe (5M * SCALE_FACTOR)
# ============================================================
NUM_WORK_ORDERS = int(5_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_work_orders ({NUM_WORK_ORDERS:,} zleceń)...')

# Lokalizacje z warsztatem
workshop_locs = df_locations[df_locations['typ'].isin(['warsztat', 'warsztat_i_sklep'])]['location_id'].values

def generate_work_orders_chunk(chunk_size, offset):
    dates = seasonal_dates(DATE_START, DATE_END, chunk_size)
    loc_ids = np.random.choice(workshop_locs, size=chunk_size)
    
    # Przypisz mechanika z danej lokalizacji
    mech_ids = []
    for lid in loc_ids:
        mechs = loc_mechanics.get(lid, mechanic_ids[:5])
        mech_ids.append(random.choice(mechs))
    
    return pd.DataFrame({
        'work_order_id': np.arange(offset + 1, offset + chunk_size + 1),
        'work_order_code': [f'WO-{i:08d}' for i in range(offset + 1, offset + chunk_size + 1)],
        'location_id': loc_ids,
        'customer_id': np.random.choice(customer_ids, size=chunk_size),
        'vehicle_id': np.random.choice(vehicle_ids, size=chunk_size),
        'mechanic_id': mech_ids,
        'data_przyjecia': dates,
        'data_zakonczenia': dates + pd.to_timedelta(np.random.randint(0, 5, size=chunk_size), unit='D'),
        'status': np.random.choice(STATUSY_ZLECEN, size=chunk_size, p=STATUSY_WAGI),
        'przebieg_przy_przyjęciu': np.random.randint(10000, 350000, size=chunk_size),
        'uwagi_klienta': np.random.choice(
            ['', 'Stuk przy hamowaniu', 'Silnik traci moc', 'Wyciek oleju',
             'Wymiana opon sezonowa', 'Przegląd okresowy', 'Klima nie chłodzi',
             'Kontrolka silnika', 'Hałas z zawieszenia', 'Wymiana klocków',
             'Przygotowanie do przeglądu', 'Wymiana oleju', 'Problem z rozrusznikiem',
             'Drgania kierownicy', 'Wymiana świec', ''],
            size=chunk_size
        ),
        'rok': dates.year,
        'miesiac': dates.month,
    })

save_table_chunked(generate_work_orders_chunk, 'fact_work_orders', NUM_WORK_ORDERS, 
                   partition_cols=['rok', 'miesiac'])

In [ ]:
# ============================================================
# fact_work_order_items - pozycje zleceń (15M * SCALE_FACTOR)
# Każde zlecenie ma 1-6 pozycji (usługa + ewentualnie części)
# ============================================================
NUM_WO_ITEMS = int(15_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_work_order_items ({NUM_WO_ITEMS:,} pozycji)...')

def generate_wo_items_chunk(chunk_size, offset):
    wo_ids = np.random.randint(1, NUM_WORK_ORDERS + 1, size=chunk_size)
    # Losowy typ pozycji: usługa lub część
    typ_pozycji = np.random.choice(['usluga', 'czesc'], size=chunk_size, p=[0.4, 0.6])
    
    srv_ids = np.where(
        typ_pozycji == 'usluga',
        np.random.choice(service_ids, size=chunk_size),
        0
    )
    prod_ids = np.where(
        typ_pozycji == 'czesc',
        np.random.choice(product_ids, size=chunk_size),
        0
    )
    
    ilosc = np.where(typ_pozycji == 'usluga', 1, np.random.randint(1, 5, size=chunk_size))
    cena_netto = np.where(
        typ_pozycji == 'usluga',
        np.random.uniform(30, 2000, size=chunk_size),
        np.random.uniform(5, 500, size=chunk_size)
    )
    cena_netto = np.round(cena_netto, 2)
    
    return pd.DataFrame({
        'wo_item_id': np.arange(offset + 1, offset + chunk_size + 1),
        'work_order_id': wo_ids,
        'typ_pozycji': typ_pozycji,
        'service_id': srv_ids.astype(int),
        'product_id': prod_ids.astype(int),
        'ilosc': ilosc,
        'cena_jednostkowa_netto': cena_netto,
        'wartosc_netto': np.round(cena_netto * ilosc, 2),
        'vat_procent': 23,
        'wartosc_brutto': np.round(cena_netto * ilosc * 1.23, 2),
        'rabat_procent': np.random.choice([0, 0, 0, 5, 10, 15], size=chunk_size),
    })

save_table_chunked(generate_wo_items_chunk, 'fact_work_order_items', NUM_WO_ITEMS)

## 4. Tabele faktowe - sprzedaż sklepowa

In [ ]:
# ============================================================
# fact_sales_transactions - transakcje sprzedaży sklepowej (30M * SCALE_FACTOR)
# ============================================================
NUM_SALES = int(30_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_sales_transactions ({NUM_SALES:,} transakcji)...')

# Lokalizacje ze sklepem
shop_locs = df_locations[df_locations['typ'].isin(['sklep', 'warsztat_i_sklep'])]['location_id'].values

def generate_sales_chunk(chunk_size, offset):
    dates = seasonal_dates(DATE_START, DATE_END, chunk_size)
    hours = np.random.choice(range(7, 20), size=chunk_size, 
                              p=[0.03, 0.08, 0.10, 0.10, 0.09, 0.08, 0.08,
                                 0.08, 0.08, 0.08, 0.08, 0.07, 0.05])
    minutes = np.random.randint(0, 60, size=chunk_size)
    
    timestamps = dates + pd.to_timedelta(hours, unit='h') + pd.to_timedelta(minutes, unit='m')
    loc_ids = np.random.choice(shop_locs, size=chunk_size)
    
    seller_arr = []
    for lid in loc_ids:
        sellers = loc_sellers.get(lid, seller_ids[:3])
        seller_arr.append(random.choice(sellers))
    
    # ~70% transakcji ma klienta zarejestrowanego, ~30% to walk-in
    has_customer = np.random.random(size=chunk_size) < 0.7
    cust_ids = np.where(has_customer, np.random.choice(customer_ids, size=chunk_size), 0)
    
    return pd.DataFrame({
        'transaction_id': np.arange(offset + 1, offset + chunk_size + 1),
        'transaction_code': [f'TRX-{i:09d}' for i in range(offset + 1, offset + chunk_size + 1)],
        'location_id': loc_ids,
        'customer_id': cust_ids,
        'employee_id': seller_arr,
        'data_transakcji': timestamps,
        'metoda_platnosci': np.random.choice(METODY_PLATNOSCI, size=chunk_size, p=PLATNOSC_WAGI),
        'nr_paragonu': [f'PAR/{random.randint(1,999):03d}/{i+offset+1:08d}' for i in range(chunk_size)],
        'rok': dates.year,
        'miesiac': dates.month,
    })

save_table_chunked(generate_sales_chunk, 'fact_sales_transactions', NUM_SALES,
                   partition_cols=['rok', 'miesiac'])

In [ ]:
# ============================================================
# fact_sales_items - pozycje sprzedaży (90M * SCALE_FACTOR)
# Średnio 3 pozycje na transakcję
# ============================================================
NUM_SALES_ITEMS = int(90_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_sales_items ({NUM_SALES_ITEMS:,} pozycji)...')

def generate_sales_items_chunk(chunk_size, offset):
    trx_ids = np.random.randint(1, NUM_SALES + 1, size=chunk_size)
    prod_ids_chunk = np.random.choice(product_ids, size=chunk_size)
    ilosc = np.random.choice([1, 1, 1, 2, 2, 3, 4], size=chunk_size)
    cena_netto = np.round(np.random.uniform(3, 600, size=chunk_size), 2)
    rabat = np.random.choice([0, 0, 0, 0, 5, 10, 15, 20], size=chunk_size)
    wartosc_po_rabacie = np.round(cena_netto * ilosc * (1 - rabat/100), 2)
    
    return pd.DataFrame({
        'sales_item_id': np.arange(offset + 1, offset + chunk_size + 1),
        'transaction_id': trx_ids,
        'product_id': prod_ids_chunk,
        'ilosc': ilosc,
        'cena_jednostkowa_netto': cena_netto,
        'rabat_procent': rabat,
        'wartosc_netto': wartosc_po_rabacie,
        'vat_procent': 23,
        'wartosc_brutto': np.round(wartosc_po_rabacie * 1.23, 2),
    })

save_table_chunked(generate_sales_items_chunk, 'fact_sales_items', NUM_SALES_ITEMS)

## 5. Tabele faktowe - faktury, płatności, magazyn

In [ ]:
# ============================================================
# fact_invoices - faktury (35M * SCALE_FACTOR)
# Faktury powiązane z work_orders i sales_transactions
# ============================================================
NUM_INVOICES = int(35_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_invoices ({NUM_INVOICES:,} faktur)...')

def generate_invoices_chunk(chunk_size, offset):
    dates = seasonal_dates(DATE_START, DATE_END, chunk_size)
    
    # ~15% faktur to faktury za zlecenia warsztatowe, ~85% za sprzedaż sklepową
    source_type = np.random.choice(
        ['work_order', 'sales'], size=chunk_size, p=[0.15, 0.85]
    )
    source_ids = np.where(
        source_type == 'work_order',
        np.random.randint(1, max(NUM_WORK_ORDERS, 1) + 1, size=chunk_size),
        np.random.randint(1, max(NUM_SALES, 1) + 1, size=chunk_size)
    )
    
    wartosc_netto = np.round(np.random.lognormal(mean=4.5, sigma=1.0, size=chunk_size), 2)
    wartosc_netto = np.clip(wartosc_netto, 10, 50000)
    wartosc_vat = np.round(wartosc_netto * 0.23, 2)
    
    # Typ: faktura VAT, paragon, faktura korygująca
    typ_dokumentu = np.random.choice(
        ['faktura_VAT', 'paragon', 'faktura_korygujaca'],
        size=chunk_size, p=[0.35, 0.60, 0.05]
    )
    
    return pd.DataFrame({
        'invoice_id': np.arange(offset + 1, offset + chunk_size + 1),
        'invoice_code': [f'FV/{dates[i].year}/{i+offset+1:08d}' for i in range(chunk_size)],
        'typ_dokumentu': typ_dokumentu,
        'source_type': source_type,
        'source_id': source_ids,
        'customer_id': np.random.choice(customer_ids, size=chunk_size),
        'location_id': np.random.randint(1, NUM_LOCATIONS + 1, size=chunk_size),
        'data_wystawienia': dates,
        'data_sprzedazy': dates - pd.to_timedelta(np.random.randint(0, 3, size=chunk_size), unit='D'),
        'termin_platnosci': dates + pd.to_timedelta(
            np.random.choice([0, 7, 14, 30], size=chunk_size, p=[0.5, 0.15, 0.2, 0.15]), unit='D'
        ),
        'wartosc_netto': wartosc_netto,
        'wartosc_vat': wartosc_vat,
        'wartosc_brutto': np.round(wartosc_netto + wartosc_vat, 2),
        'status': np.random.choice(
            ['oplacona', 'oczekuje', 'przeterminowana', 'anulowana'],
            size=chunk_size, p=[0.80, 0.10, 0.07, 0.03]
        ),
        'rok': dates.year,
        'miesiac': dates.month,
    })

save_table_chunked(generate_invoices_chunk, 'fact_invoices', NUM_INVOICES,
                   partition_cols=['rok', 'miesiac'])

In [ ]:
# ============================================================
# fact_payments - płatności (35M * SCALE_FACTOR)
# ============================================================
NUM_PAYMENTS = int(35_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_payments ({NUM_PAYMENTS:,} płatności)...')

def generate_payments_chunk(chunk_size, offset):
    dates = seasonal_dates(DATE_START, DATE_END, chunk_size)
    kwota = np.round(np.random.lognormal(mean=4.5, sigma=1.0, size=chunk_size), 2)
    kwota = np.clip(kwota, 5, 60000)
    
    return pd.DataFrame({
        'payment_id': np.arange(offset + 1, offset + chunk_size + 1),
        'invoice_id': np.random.randint(1, max(NUM_INVOICES, 1) + 1, size=chunk_size),
        'data_platnosci': dates,
        'kwota': kwota,
        'metoda_platnosci': np.random.choice(METODY_PLATNOSCI, size=chunk_size, p=PLATNOSC_WAGI),
        'status': np.random.choice(
            ['zrealizowana', 'oczekuje', 'odrzucona', 'zwrot'],
            size=chunk_size, p=[0.90, 0.05, 0.03, 0.02]
        ),
        'numer_transakcji': [f'PAY-{uuid.uuid4().hex[:12].upper()}' for _ in range(chunk_size)],
        'rok': dates.year,
        'miesiac': dates.month,
    })

save_table_chunked(generate_payments_chunk, 'fact_payments', NUM_PAYMENTS,
                   partition_cols=['rok', 'miesiac'])

In [ ]:
# ============================================================
# fact_inventory_movements - ruchy magazynowe (50M * SCALE_FACTOR)
# ============================================================
NUM_INVENTORY = int(50_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_inventory_movements ({NUM_INVENTORY:,} ruchów)...')

def generate_inventory_chunk(chunk_size, offset):
    dates = random_dates(DATE_START, DATE_END, chunk_size)
    
    typ_ruchu = np.random.choice(
        ['przyjecie', 'wydanie_sprzedaz', 'wydanie_warsztat', 'zwrot', 'korekta', 'inwentaryzacja'],
        size=chunk_size, p=[0.25, 0.35, 0.25, 0.05, 0.05, 0.05]
    )
    
    ilosc = np.random.randint(1, 20, size=chunk_size)
    # Wydania mają ujemną ilość
    ilosc = np.where(
        np.isin(typ_ruchu, ['wydanie_sprzedaz', 'wydanie_warsztat']),
        -ilosc, ilosc
    )
    
    return pd.DataFrame({
        'movement_id': np.arange(offset + 1, offset + chunk_size + 1),
        'product_id': np.random.choice(product_ids, size=chunk_size),
        'location_id': np.random.randint(1, NUM_LOCATIONS + 1, size=chunk_size),
        'typ_ruchu': typ_ruchu,
        'ilosc': ilosc,
        'data_ruchu': dates,
        'dokument_zrodlowy': np.random.choice(
            ['PZ', 'WZ', 'RW', 'ZW', 'KOR', 'INW'],
            size=chunk_size
        ),
        'nr_dokumentu': [f'DOC-{i+offset+1:09d}' for i in range(chunk_size)],
        'wartosc_netto': np.round(np.abs(ilosc) * np.random.uniform(5, 500, size=chunk_size), 2),
        'uwagi': np.random.choice(['', '', '', 'Dostawa regularna', 'Zamówienie specjalne',
                                     'Zwrot od klienta', 'Korekta stanów', ''], size=chunk_size),
        'rok': dates.year,
        'miesiac': dates.month,
    })

save_table_chunked(generate_inventory_chunk, 'fact_inventory_movements', NUM_INVENTORY,
                   partition_cols=['rok', 'miesiac'])

## 6. Tabele wspierające

In [ ]:
# ============================================================
# fact_appointments - rezerwacje wizyt (5M * SCALE_FACTOR)
# ============================================================
NUM_APPOINTMENTS = int(5_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_appointments ({NUM_APPOINTMENTS:,} rezerwacji)...')

def generate_appointments_chunk(chunk_size, offset):
    dates = seasonal_dates(DATE_START, DATE_END, chunk_size)
    hours = np.random.choice(range(7, 17), size=chunk_size)
    timestamps = dates + pd.to_timedelta(hours, unit='h')
    
    return pd.DataFrame({
        'appointment_id': np.arange(offset + 1, offset + chunk_size + 1),
        'customer_id': np.random.choice(customer_ids, size=chunk_size),
        'vehicle_id': np.random.choice(vehicle_ids, size=chunk_size),
        'location_id': np.random.choice(workshop_locs, size=chunk_size),
        'service_id': np.random.choice(service_ids, size=chunk_size),
        'data_rezerwacji': dates - pd.to_timedelta(np.random.randint(1, 14, size=chunk_size), unit='D'),
        'data_wizyty': timestamps,
        'status': np.random.choice(
            ['potwierdzona', 'zrealizowana', 'anulowana', 'niestawienie_sie'],
            size=chunk_size, p=[0.10, 0.75, 0.10, 0.05]
        ),
        'kanal_rezerwacji': np.random.choice(
            ['telefon', 'online', 'osobiscie', 'email'],
            size=chunk_size, p=[0.35, 0.40, 0.15, 0.10]
        ),
        'uwagi': np.random.choice(
            ['', '', '', 'Proszę o kontakt telefoniczny', 'Samochód zastępczy',
             'Preferuję rano', 'Pilne', 'Umówiony wcześniej', ''],
            size=chunk_size
        ),
        'rok': dates.year,
        'miesiac': dates.month,
    })

save_table_chunked(generate_appointments_chunk, 'fact_appointments', NUM_APPOINTMENTS,
                   partition_cols=['rok', 'miesiac'])

In [ ]:
# ============================================================
# fact_purchase_orders - zamówienia do dostawców (500K * SCALE_FACTOR)
# ============================================================
NUM_PO = int(500_000 * SCALE_FACTOR)
print(f'Generowanie fact_purchase_orders ({NUM_PO:,} zamówień)...')

def generate_po_chunk(chunk_size, offset):
    dates = random_dates(DATE_START, DATE_END, chunk_size)
    wartosc = np.round(np.random.lognormal(mean=7, sigma=0.8, size=chunk_size), 2)
    wartosc = np.clip(wartosc, 200, 100000)
    
    return pd.DataFrame({
        'po_id': np.arange(offset + 1, offset + chunk_size + 1),
        'po_code': [f'PO-{i+offset+1:07d}' for i in range(chunk_size)],
        'supplier_id': np.random.choice(supplier_ids, size=chunk_size),
        'location_id': np.random.randint(1, NUM_LOCATIONS + 1, size=chunk_size),
        'data_zamowienia': dates,
        'data_dostawy_planowana': dates + pd.to_timedelta(np.random.randint(3, 21, size=chunk_size), unit='D'),
        'data_dostawy_rzeczywista': dates + pd.to_timedelta(np.random.randint(3, 25, size=chunk_size), unit='D'),
        'wartosc_netto': wartosc,
        'wartosc_brutto': np.round(wartosc * 1.23, 2),
        'status': np.random.choice(
            ['zlozono', 'w_realizacji', 'dostarczone', 'czesciowo_dostarczone', 'anulowane'],
            size=chunk_size, p=[0.03, 0.05, 0.85, 0.05, 0.02]
        ),
        'rok': dates.year,
    })

save_table_chunked(generate_po_chunk, 'fact_purchase_orders', NUM_PO)

# ============================================================
# fact_purchase_order_items - pozycje zamówień (2M * SCALE_FACTOR)
# ============================================================
NUM_PO_ITEMS = int(2_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_purchase_order_items ({NUM_PO_ITEMS:,} pozycji)...')

def generate_po_items_chunk(chunk_size, offset):
    ilosc = np.random.randint(1, 50, size=chunk_size)
    cena = np.round(np.random.uniform(5, 500, size=chunk_size), 2)
    
    return pd.DataFrame({
        'po_item_id': np.arange(offset + 1, offset + chunk_size + 1),
        'po_id': np.random.randint(1, max(NUM_PO, 1) + 1, size=chunk_size),
        'product_id': np.random.choice(product_ids, size=chunk_size),
        'ilosc_zamowiona': ilosc,
        'ilosc_dostarczona': np.clip(ilosc + np.random.randint(-2, 1, size=chunk_size), 0, 100),
        'cena_jednostkowa_netto': cena,
        'wartosc_netto': np.round(cena * ilosc, 2),
    })

save_table_chunked(generate_po_items_chunk, 'fact_purchase_order_items', NUM_PO_ITEMS)

In [ ]:
# ============================================================
# fact_customer_feedback - opinie klientów (2M * SCALE_FACTOR)
# ============================================================
NUM_FEEDBACK = int(2_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_customer_feedback ({NUM_FEEDBACK:,} opinii)...')

KOMENTARZE = [
    'Bardzo profesjonalna obsługa', 'Szybka realizacja', 'Polecam!',
    'Trochę za drogo', 'Długi czas oczekiwania', 'Świetna komunikacja',
    'Fachowa naprawa', 'Samochód gotowy przed terminem', 'Miła obsługa',
    'Mogłoby być taniej', 'Wrócę na pewno', 'Solidna robota',
    'Uczciwe ceny', 'Problem wrócił po miesiącu', 'Brak uwag',
    'Rewelacja!', 'Przeciętnie', 'Do poprawy', 'OK', '',
]

def generate_feedback_chunk(chunk_size, offset):
    dates = random_dates(DATE_START, DATE_END, chunk_size)
    
    # Oceny z rozkładem: więcej pozytywnych
    oceny = np.random.choice(
        [1, 2, 3, 4, 5], size=chunk_size,
        p=[0.03, 0.05, 0.12, 0.30, 0.50]
    )
    
    return pd.DataFrame({
        'feedback_id': np.arange(offset + 1, offset + chunk_size + 1),
        'customer_id': np.random.choice(customer_ids, size=chunk_size),
        'location_id': np.random.randint(1, NUM_LOCATIONS + 1, size=chunk_size),
        'work_order_id': np.random.randint(1, max(NUM_WORK_ORDERS, 1) + 1, size=chunk_size),
        'data_opinii': dates,
        'ocena': oceny,
        'komentarz': np.random.choice(KOMENTARZE, size=chunk_size),
        'kategoria': np.random.choice(
            ['obsluga', 'jakosc_naprawy', 'czas_realizacji', 'cena', 'czystosc', 'ogolna'],
            size=chunk_size, p=[0.20, 0.25, 0.15, 0.15, 0.10, 0.15]
        ),
        'kanal': np.random.choice(
            ['google', 'formularz_online', 'email', 'telefon'],
            size=chunk_size, p=[0.40, 0.30, 0.20, 0.10]
        ),
    })

save_table_chunked(generate_feedback_chunk, 'fact_customer_feedback', NUM_FEEDBACK)

In [ ]:
# ============================================================
# fact_loyalty_program - program lojalnościowy (500K * SCALE_FACTOR)
# ============================================================
NUM_LOYALTY = int(500_000 * SCALE_FACTOR)
print(f'Generowanie fact_loyalty_program ({NUM_LOYALTY:,} wpisów)...')

def generate_loyalty_chunk(chunk_size, offset):
    dates = random_dates(date(2021, 1, 1), DATE_END, chunk_size)  # Program od 2021
    
    return pd.DataFrame({
        'loyalty_id': np.arange(offset + 1, offset + chunk_size + 1),
        'customer_id': np.random.choice(customer_ids, size=chunk_size),
        'data_zdarzenia': dates,
        'typ_zdarzenia': np.random.choice(
            ['naliczenie_punktow', 'wymiana_punktow', 'bonus', 'wygasniecie'],
            size=chunk_size, p=[0.60, 0.20, 0.10, 0.10]
        ),
        'punkty': np.random.choice(
            [-500, -200, -100, 10, 20, 50, 100, 200, 500],
            size=chunk_size, p=[0.05, 0.07, 0.08, 0.20, 0.25, 0.15, 0.10, 0.05, 0.05]
        ),
        'opis': np.random.choice(
            ['Zakup w sklepie', 'Usługa warsztatowa', 'Bonus powitalny',
             'Bonus urodzinowy', 'Wymiana na rabat 10%', 'Wymiana na rabat 20%',
             'Wymiana na darmowy przegląd', 'Punkty wygasłe', 'Polecenie znajomego'],
            size=chunk_size
        ),
        'saldo_po': np.random.randint(0, 5000, size=chunk_size),
        'poziom': np.random.choice(
            ['standard', 'silver', 'gold', 'platinum'],
            size=chunk_size, p=[0.50, 0.30, 0.15, 0.05]
        ),
    })

save_table_chunked(generate_loyalty_chunk, 'fact_loyalty_program', NUM_LOYALTY)

In [ ]:
# ============================================================
# fact_employee_schedules - grafiki pracy (3M * SCALE_FACTOR)
# ============================================================
NUM_SCHEDULES = int(3_000_000 * SCALE_FACTOR)
print(f'Generowanie fact_employee_schedules ({NUM_SCHEDULES:,} wpisów)...')

all_employee_ids = df_employees['employee_id'].values

def generate_schedules_chunk(chunk_size, offset):
    dates = random_dates(DATE_START, DATE_END, chunk_size)
    
    godzina_start = np.random.choice([6, 7, 8, 9, 10, 12, 14], size=chunk_size,
                                       p=[0.05, 0.25, 0.30, 0.15, 0.05, 0.10, 0.10])
    czas_pracy = np.random.choice([4, 6, 8, 10, 12], size=chunk_size,
                                    p=[0.10, 0.10, 0.60, 0.15, 0.05])
    
    return pd.DataFrame({
        'schedule_id': np.arange(offset + 1, offset + chunk_size + 1),
        'employee_id': np.random.choice(all_employee_ids, size=chunk_size),
        'data': dates,
        'godzina_start': godzina_start,
        'godzina_koniec': godzina_start + czas_pracy,
        'typ_zmiany': np.random.choice(
            ['dzienna', 'poranna', 'popoludniowa', 'nocna', 'wolne', 'urlop', 'chorobowe'],
            size=chunk_size, p=[0.40, 0.15, 0.15, 0.02, 0.15, 0.08, 0.05]
        ),
        'nadgodziny_h': np.random.choice(
            [0, 0, 0, 0, 0, 1, 2, 3, 4],
            size=chunk_size
        ),
        'obecnosc': np.random.choice(
            ['obecny', 'nieobecny_usprawiedliwiony', 'nieobecny_nieusprawiedliwiony', 'spozniony'],
            size=chunk_size, p=[0.88, 0.08, 0.02, 0.02]
        ),
    })

save_table_chunked(generate_schedules_chunk, 'fact_employee_schedules', NUM_SCHEDULES)

## 7. Walidacja i statystyki

In [ ]:
# ============================================================
# WALIDACJA I PODSUMOWANIE
# ============================================================
import glob as glob_module

print('=' * 60)
print('PODSUMOWANIE GENERACJI DANYCH')
print('=' * 60)
print(f'SCALE_FACTOR: {SCALE_FACTOR}')
print(f'Katalog: {os.path.abspath(OUTPUT_DIR)}')
print()

total_size = 0
table_stats = []

for table_dir in sorted(os.listdir(OUTPUT_DIR)):
    table_path = os.path.join(OUTPUT_DIR, table_dir)
    if os.path.isdir(table_path):
        # Zlicz rozmiar plików
        size = 0
        file_count = 0
        for root, dirs, files in os.walk(table_path):
            for f in files:
                fp = os.path.join(root, f)
                size += os.path.getsize(fp)
                file_count += 1
        
        size_mb = size / (1024 * 1024)
        total_size += size
        table_stats.append({
            'tabela': table_dir,
            'pliki': file_count,
            'rozmiar_MB': round(size_mb, 1),
        })

df_stats = pd.DataFrame(table_stats)
print(df_stats.to_string(index=False))
print()
print(f'ŁĄCZNY ROZMIAR: {total_size / (1024**3):.2f} GB')
print(f'Szacowany rozmiar przy SCALE_FACTOR=1.0: ~{total_size / (1024**3) / SCALE_FACTOR:.1f} GB')
print()
print('Gotowe! Dane znajdują się w katalogu:', os.path.abspath(OUTPUT_DIR))
print()
print('Aby załadować dane do Databricks:')
print('  1. Wgraj katalog output_data/ do DBFS lub Unity Catalog Volume')
print('  2. Użyj spark.read.parquet("dbfs:/path/to/tabela/") ')
print('  3. Lub CREATE TABLE ... USING PARQUET LOCATION ...')